In [1]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 데이터 로드
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

# 결측값 처리
train.fillna('WT', inplace=True)
test.fillna('WT', inplace=True)

# 변이 열 준비
mutation_columns = [col for col in train.columns if col not in ['ID', 'SUBCLASS']]

# 변이 정보를 문자열로 결합
train['mutations'] = train[mutation_columns].apply(lambda x: ' '.join(x.astype(str)), axis=1)
test['mutations'] = test[mutation_columns].apply(lambda x: ' '.join(x.astype(str)), axis=1)

# 타겟 레이블 인코딩
label_encoder = LabelEncoder()
train['SUBCLASS'] = label_encoder.fit_transform(train['SUBCLASS'])
num_classes = len(label_encoder.classes_)

# 특징과 타겟 준비
X = train['mutations']
y = train['SUBCLASS']

# 텍스트 데이터를 숫자 특징으로 변환하기 위해 TF-IDF 벡터화를 사용
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

X_tfidf = tfidf.fit_transform(X)
X_test_tfidf = tfidf.transform(test['mutations'])

# 학습 데이터를 훈련 세트와 검증 세트로 분할
X_train, X_valid, y_train, y_valid = train_test_split(X_tfidf, y, test_size=0.2, random_state=42, stratify=y)


/tmp/ipykernel_57774/3742414261.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test['mutations'] = test[mutation_columns].apply(lambda x: ' '.join(x.astype(str)), axis=1)


In [3]:
# Auto-sklearn 임포트
import autosklearn.classification
from sklearn.metrics import classification_report

# Auto-sklearn 분류기 생성
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=3600,  # 총 시간 제한 (초)
    per_run_time_limit=360,  # 각 모델당 시간 제한 (초)
    metric=autosklearn.metrics.log_loss,
    n_jobs=-1,
    memory_limit=10240  # 메모리 제한 (MB)
)

# AutoML 모델 학습
automl.fit(X_train, y_train)

# 검증 데이터에 대한 예측
y_pred = automl.predict(X_valid)

# 성능 평가
print("Auto-sklearn Classification Report:")
print(classification_report(y_valid, y_pred, target_names=label_encoder.classes_))

# 테스트 데이터에 대한 예측
y_test_pred = automl.predict(X_test_tfidf)
test['SUBCLASS'] = label_encoder.inverse_transform(y_test_pred)

# 제출 파일 생성
submission = test[['ID', 'SUBCLASS']]
submission.to_csv('submission_autosklearn.csv', index=False)
print("Auto-sklearn 제출 파일이 생성되었습니다: submission_autosklearn.csv")


[WARNING] [2024-10-15 07:02:17,690:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2024-10-15 07:02:19,173:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
Auto-sklearn Classification Report:
              precision    recall  f1-score   support

         ACC       1.00      0.79      0.88        14
        BLCA       0.10      0.05      0.06        21
        BRCA       0.26      0.52      0.35       157
        CESC       0.00      0.00      0.00        31
        COAD       0.55      0.13      0.21        45
        DLBC       0.00      0.00      0.00         7
      GBMLGG       0.32      0.25      0.28        92
        HNSC       0.09      0.02      0.04        45
       KIPAN       0.17      0.58      0.26       103
        KIRC       0.00      0.00      0.00        67
        LAML       0.29      0.31      0.30        32
         LGG       0.29      0.22      0.

/opt/conda/envs/autosklearn2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/autosklearn2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/autosklearn2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Auto-sklearn 제출 파일이 생성되었습니다: submission_autosklearn.csv


/tmp/ipykernel_57774/1405901231.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test['SUBCLASS'] = label_encoder.inverse_transform(y_test_pred)
